In [39]:
import pandas as pd
from scraper import search_options, page_driver
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# drop unnamed columns 
# drop duplicates
# drop 'Proj. Time' column
# drop any 'Duo' 

# questions
# should I convert to seconds or minutes?
# does the sub division matter? VI, MI, 

In [33]:
url = 'http://registration.baa.org/2018/cf/Public/iframe_ResultsSearch.cfm?mode=entry'
html = page_driver(url)
divisions, gender, states, countries = search_options(html)
divisions

['18-39 Age Group',
 '40-44 Age Group',
 '45-49 Age Group',
 '50-54 Age Group',
 '55-59 Age Group',
 '60-64 Age Group',
 '65-69 Age Group',
 '70-74 Age Group',
 '75-79 Age Group',
 '80+ Age Group']

In [44]:
data_storage = {'Bib': [],
             'Name': [],
             'Age': [],
             'M/F': [],
             'City':[],
             'State': [],
             'Country': [],
             'Citizen': [],
             'Blank': [],
             '5K': [],
             '10K': [],
             '15K': [],
             '20K': [],
             'Half': [],
             '25K': [],
             '30K': [],
             '35K': [],
             '40K': [],
             'Pace': [],
             'Proj. Time': [],
             'Official Time': [],
             'Overall': [],
             'Gender': [],
             'Division': []
            }


In [43]:
def scrape_results(page):
    '''This function takes the html from the page, extracts
    the table data and stores it in an external dictionary.
    '''

    soup = BeautifulSoup(page, "lxml")

    info_headers = ['Bib', 'Name', 'Age', 'M/F', 'City', 'State',
                    'Country', 'Citizen', 'Blank']

    result_headers = ['5K', '10K', '15K', '20K', 'Half', '25K',
                      '30K', '35K', '40K', 'Pace', 'Proj. Time',
                      'Official Time', 'Overall', 'Gender', 'Division']

    # scrape each participants info
    racer_info = soup.findAll("tr", {"class": "tr_header"})
    for i, racer in enumerate(racer_info):
        for key, val in zip(info_headers, racer_info[i].find_all('td')):
            data_storage[key] += [val.text.strip()]

    # scrape each participants results
    race_times = soup.findAll("table", {"class": "table_infogrid"})
    for i, time in enumerate(race_times):
        for key, val in zip(result_headers, race_times[i].find_all('td')):
            data_storage[key] += [val.text.strip()]

def scrape_by_state(states, divisions):
    '''Scrape the race results based on the state runner is from.'''

    url = 'http://registration.baa.org/2018/cf/Public/iframe_ResultsSearch.cfm'
    driver = webdriver.Chrome()
    driver.get(url)
    max_limit = 1000

    for state in ['Massachusetts', 'California', 'New York', 'Texas']:
        for division in divisions:
            division_option = driver.find_element_by_name("AwardsDivisionID")
            division_option.send_keys(division)
            state_option = driver.find_element_by_name("StateID")
            state_option.send_keys(state)
            limit = driver.find_element_by_name("VarTargetCount")
            limit.send_keys(max_limit)

            try:
                xpath1 = '//*[@id="PublicSearch"]/div/div/input'
                submit_button = driver.find_elements_by_xpath(xpath1)[0]
                time.sleep(3)
                submit_button.click()
                scrape_results(driver.page_source)
            except IndexError:
                xpath4 = '/html/body/div/div/div/div/table[2]/tbody/tr/td[1]/form/input'
                search_again = driver.find_elements_by_xpath(xpath4)[0]
                time.sleep(3)
                search_again.click()
                continue

            try:
                xpath2 = ('/html/body/div/div/div/div/table[4]/tbody/tr/td/'
                          'table/tbody/tr[51]/td/table/tbody/tr/td[2]/form/input[2]')
                next_button = driver.find_elements_by_xpath(xpath2)[0]
                time.sleep(3)
                next_button.click()
                scrape_results(driver.page_source)
            except IndexError:
                xpath4 = '/html/body/div/div/div/div/table[2]/tbody/tr/td[1]/form/input'
                search_again = driver.find_elements_by_xpath(xpath4)[0]
                time.sleep(3)
                search_again.click()
                continue

            xpath3 = ('/html/body/div/div/div/div/table[3]/tbody/tr/td/table/tbody'
                      '/tr[51]/td/table/tbody/tr/td[2]/form/input[2]')
            while True:
                try:
                    next_button = driver.find_elements_by_xpath(xpath3)[0]
                    time.sleep(3)
                    next_button.click()
                    scrape_results(driver.page_source)
                except IndexError:
                    xpath4 = '/html/body/div/div/div/div/table[2]/tbody/tr/td[1]/form/input'
                    search_again = driver.find_elements_by_xpath(xpath4)[0]
                    time.sleep(3)
                    search_again.click()
                    break

In [46]:
# states = ['Massachusetts', 'California', 'New York', 'Texas']
# scrape_by_state(states, divisions)
supplement = pd.DataFrame(data_storage)
supplement.to_csv('supplement_2018.csv')
df18_extra = pd.read_csv('supplement_2018.csv')
print(df18_extra.shape)
df18_extra.head()

(6841, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Blank,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,48,"Vassallo, Daniel",32,M,Peabody,MA,USA,NaN,NaN,...,1:24:41,1:42:37,2:00:56,2:19:23,0:05:39,NaN,2:27:50,10,10,10
1,1,248,"Herzig, Matthew",22,M,Brookline,MA,USA,NaN,NaN,...,1:24:45,1:42:29,2:00:45,2:19:23,0:05:39,NaN,2:27:55,12,12,12
2,2,46,"Ayr, Jason",30,M,Brighton,MA,USA,NaN,NaN,...,1:24:41,-,2:00:58,2:20:41,0:05:43,NaN,2:29:53,22,22,21
3,3,51,"Mindel, Scott",31,M,Burlington,MA,USA,NaN,NaN,...,1:25:48,1:44:01,2:02:40,2:22:41,0:05:50,NaN,2:32:48,30,30,29
4,4,137,"Zhou, Fan",29,M,Boston,MA,USA,NaN,NaN,...,1:29:32,1:47:44,2:06:08,2:24:28,0:05:50,NaN,2:32:50,31,31,30


In [3]:
df15 = pd.read_csv('data/marathon_results_2015.csv')
df16 = pd.read_csv('data/marathon_results_2016.csv')
df17 = pd.read_csv('data/marathon_results_2017.csv')
df18 = pd.read_csv('data/marathon_results_2018.csv')
print(df18.shape)
df18.head()

(19884, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Blank,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [13]:
# find duplicates
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19884 entries, 0 to 19883
Data columns (total 25 columns):
Unnamed: 0       19884 non-null int64
Bib              19884 non-null object
Name             19884 non-null object
Age              19884 non-null int64
M/F              19884 non-null object
City             19884 non-null object
State            15847 non-null object
Country          19884 non-null object
Citizen          1113 non-null object
Blank            85 non-null object
5K               19884 non-null object
10K              19884 non-null object
15K              19884 non-null object
20K              19884 non-null object
Half             19884 non-null object
25K              19884 non-null object
30K              19884 non-null object
35K              19884 non-null object
40K              19884 non-null object
Pace             19884 non-null object
Proj. Time       192 non-null object
Official Time    19884 non-null object
Overall          19884 non-null int64
Gen

In [25]:
# df18['Blank'].value_counts()
df18[df18['State'] == 'MA']  # .value_counts()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Blank,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
5528,5528,48,"Vassallo, Daniel",32,M,Peabody,MA,USA,NaN,NaN,...,1:24:41,1:42:37,2:00:56,2:19:23,0:05:39,NaN,2:27:50,10,10,10
5529,5529,248,"Herzig, Matthew",22,M,Brookline,MA,USA,NaN,NaN,...,1:24:45,1:42:29,2:00:45,2:19:23,0:05:39,NaN,2:27:55,12,12,12
5530,5530,46,"Ayr, Jason",30,M,Brighton,MA,USA,NaN,NaN,...,1:24:41,-,2:00:58,2:20:41,0:05:43,NaN,2:29:53,22,22,21
5531,5531,51,"Mindel, Scott",31,M,Burlington,MA,USA,NaN,NaN,...,1:25:48,1:44:01,2:02:40,2:22:41,0:05:50,NaN,2:32:48,30,30,29
5532,5532,137,"Zhou, Fan",29,M,Boston,MA,USA,NaN,NaN,...,1:29:32,1:47:44,2:06:08,2:24:28,0:05:50,NaN,2:32:50,31,31,30
5533,5533,469,"Sanseverino, Cole",29,M,Boston,MA,USA,NaN,NaN,...,1:28:25,1:46:12,2:04:01,2:22:55,0:05:51,NaN,2:32:58,33,33,32
5534,5534,563,"Lemon, Nicholas",25,M,Brighton,MA,USA,NaN,NaN,...,1:27:35,1:45:48,2:04:34,2:23:59,0:05:51,NaN,2:33:07,34,34,33
5535,5535,227,"Roosa, Nicholas",32,M,Greenfield,MA,USA,NaN,NaN,...,1:29:22,1:47:57,2:07:02,2:26:39,0:05:57,NaN,2:35:59,61,61,57
5536,5536,124,"Allen, Christopher",24,M,Norfolk,MA,USA,NaN,NaN,...,1:28:50,1:47:13,2:06:49,2:27:24,0:05:59,NaN,2:36:27,68,68,63
5537,5537,210,"Klecker, Jack",26,M,Boston,MA,USA,NaN,NaN,...,1:32:56,1:51:26,2:10:11,2:28:21,0:05:59,NaN,2:36:29,69,69,64


In [10]:
print(df17.shape)
df17.head()

(26410, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1,1
1,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2,2
2,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,1:17:00,1:33:01,1:48:31,2:03:38,0:04:59,-,2:10:28,3,3,3
3,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,1:17:00,1:33:01,1:48:58,2:04:35,0:05:03,-,2:12:08,4,4,4
4,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,1:16:59,1:33:01,1:48:41,2:05:00,0:05:04,-,2:12:35,5,5,5


In [26]:
df17[df17['State'] == 'MA']

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
10,10,22,"Quigley, Sean",32,M,Braintree,MA,USA,NaN,NaN,...,1:18:20,1:34:58,1:51:55,2:08:10,0:05:11,-,2:15:34,11,11,10
19,19,31,"Harvey, Brian",29,M,Boston,MA,USA,NaN,NaN,...,1:21:45,1:38:34,1:55:48,2:12:51,0:05:22,-,2:20:18,20,20,17
22,22,101,"Mindel, Scott",30,M,Arlington,MA,USA,NaN,NaN,...,1:23:19,1:40:09,1:57:20,2:14:34,0:05:27,-,2:22:32,23,22,18
25,25,182,"Place, Ryan J",31,M,Cambridge,MA,USA,NaN,NaN,...,1:23:13,1:40:12,1:57:40,2:15:14,0:05:28,-,2:23:05,26,23,19
27,27,28,"Ashe, Eric",28,M,Boston,MA,USA,NaN,NaN,...,1:23:29,1:40:14,1:57:24,2:15:27,0:05:29,-,2:23:26,28,25,21
31,31,54,"Ayr, Jason M.",29,M,Allston,MA,USA,NaN,NaN,...,1:23:19,1:40:15,1:58:09,2:16:27,0:05:32,-,2:24:49,32,29,25
38,38,123,"Carpenter, Joseph K.",27,M,Cambridge,MA,USA,NaN,NaN,...,1:26:16,1:43:21,2:00:54,2:18:49,0:05:36,-,2:26:40,39,35,29
47,47,56,"Fischer, Matthew",24,M,Brighton,MA,USA,NaN,NaN,...,1:21:58,1:40:14,1:59:34,2:19:11,0:05:40,-,2:28:18,48,43,36
57,57,219,"Taylor, Alexander J.",37,M,Lincoln,MA,USA,NaN,NaN,...,1:27:17,1:46:04,2:04:26,2:22:25,0:05:44,-,2:30:19,58,52,44
65,65,248,"Gilmore, Peter",27,M,Boston,MA,USA,NaN,NaN,...,1:26:56,1:44:54,2:03:28,2:22:28,0:05:47,-,2:31:13,66,59,49


In [14]:
df17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26410 entries, 0 to 26409
Data columns (total 25 columns):
Unnamed: 0       26410 non-null int64
Bib              26410 non-null object
Name             26410 non-null object
Age              26410 non-null int64
M/F              26410 non-null object
City             26410 non-null object
State            22815 non-null object
Country          26410 non-null object
Citizen          1254 non-null object
Unnamed: 9       91 non-null object
5K               26410 non-null object
10K              26410 non-null object
15K              26410 non-null object
20K              26410 non-null object
Half             26410 non-null object
25K              26410 non-null object
30K              26410 non-null object
35K              26410 non-null object
40K              26410 non-null object
Pace             26410 non-null object
Proj Time        26410 non-null object
Official Time    26410 non-null object
Overall          26410 non-null int64
G

In [17]:
df17['Unnamed: 9'].value_counts()

VI    54
MI    37
Name: Unnamed: 9, dtype: int64

In [11]:
print(df16.shape)
df16.head()

(26630, 24)


,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 8,5K,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,5,"Hayle, Lemi Berhanu",21,M,Addis Ababa,NaN,ETH,NaN,NaN,0:15:47,...,1:19:15,1:34:17,1:50:24,2:05:59,0:05:04,2:12:45,2:12:45,1,1,1
1,1,"Desisa, Lelisa",26,M,Ambo,NaN,ETH,NaN,NaN,0:15:47,...,1:19:15,1:34:17,1:50:24,2:05:59,0:05:06,2:13:32,2:13:32,2,2,2
2,6,"Tsegay, Yemane Adhane",31,M,Addis Ababa,NaN,ETH,NaN,NaN,0:15:46,...,1:19:15,1:34:45,1:50:48,2:06:47,0:05:07,2:14:02,2:14:02,3,3,3
3,11,"Korir, Wesley",33,M,Kitale,NaN,KEN,NaN,NaN,0:15:46,...,1:19:16,1:34:45,1:50:48,2:06:47,0:05:07,2:14:05,2:14:05,4,4,4
4,14,"Lonyangata, Paul",23,M,Eldoret,NaN,KEN,NaN,NaN,0:15:46,...,1:19:18,1:34:46,1:51:30,2:08:11,0:05:11,2:15:45,2:15:45,5,5,5


In [12]:
print(df15.shape)
df15.head()

(26598, 25)


,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:02:39,0:04:56,-,2:09:17,1,1,1
1,1,4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,NaN,NaN,...,1:16:07,1:31:59,1:47:59,2:02:42,0:04:58,-,2:09:48,2,2,2
2,2,8,"Chebet, Wilson",29,M,Marakwet,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:03:01,0:04:59,-,2:10:22,3,3,3
3,3,11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:48:03,2:03:47,0:05:00,-,2:10:47,4,4,4
4,4,10,"Korir, Wesley",32,M,Kitale,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:03:27,0:05:00,-,2:10:49,5,5,5
